In [2]:
# 日付データの特徴量
import pandas as pd
# 10時間の頻度の datetime 列
s = pd.date_range("2020-01-06", "2020-01-10", freq="10H").to_series()

In [5]:
# datetime から特徴量を
features = {
    "dayofweek": s.dt.dayofweek.values,
    "dayofyear": s.dt.dayofyear.values,
    "hour": s.dt.hour.values,
    "is_leap_year": s.dt.is_leap_year.values,
    "quater": s.dt.quarter.values,
    "weekofyear": s.dt.weekofyear.values
}

In [7]:
features

{'dayofweek': array([0, 0, 0, 1, 1, 2, 2, 2, 3, 3]),
 'dayofyear': array([6, 6, 6, 7, 7, 8, 8, 8, 9, 9]),
 'hour': array([ 0, 10, 20,  6, 16,  2, 12, 22,  8, 18]),
 'is_leap_year': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]),
 'quater': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'weekofyear': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])}

In [9]:
# p.147 のデータがないため作成
example_df = pd.DataFrame([
  {
    "date": "2016-09-01",
    "customer_id": 146361,
    "cat1": 2,
    "cat2": 2,
    "cat3": 0,
    "num1": -0.518679
  },
  {
    "date": "2017-04-01",
    "customer_id": 180838,
    "cat1": 4,
    "cat2": 1,
    "cat3": 0,
    "num1": 0.415853
  },
  {
    "date": "2017-08-01",
    "customer_id": 157857,
    "cat1": 3,
    "cat2": 3,
    "cat3": 1,
    "num1": -2.061687
  },
  {
    "date": "2017-12-01",
    "customer_id": 159772,
    "cat1": 5,
    "cat2": 1,
    "cat3": 1,
    "num1": -0.276558
  },
  {
    "date": "2017-09-01",
    "customer_id": 80014,
    "cat1": 3,
    "cat2": 2,
    "cat3": 1,
    "num1": -1.456827
  }
])

In [11]:
example_df.head()

,date,customer_id,cat1,cat2,cat3,num1
0,2016-09-01,146361,2,2,0,-0.518679
1,2017-04-01,180838,4,1,0,0.415853
2,2017-08-01,157857,3,3,1,-2.061687
3,2017-12-01,159772,5,1,1,-0.276558
4,2017-09-01,80014,3,2,1,-1.456827


In [14]:
def genearate_features(df: pd.DataFrame):
    df.loc[:, "year"] = df["date"].dt.year
    df.loc[:, "weekofyear"] = df["date"].dt.weekofyear
    df.loc[:, "month"] = df["date"].dt.month
    df.loc[:, "dayofweek"] = df["date"].dt.dayofweeks
    df.loc[:, "weekend"] = (df["date"].dt.weekday >= 5).astype(int)
    
    aggs = {}
    aggs["month"] = ["nunique", "mean"]
    aggs["weekofyear"] = ["nunique", "mean"]
    aggs["num1"] = ["sum", "max", "min", "mean"]
    aggs["customer_id"] = ["size"]
    aggs["customer_id"] = ["nunique"]
    
    # customer_id を軸に集約
    agg_df = df.groupby("customer_id").agg(aggs)
    agg_df = agg_df.reset_index()
    return agg_df

In [17]:
example_df["date"] = pd.to_datetime(example_df["date"])

In [19]:
example_result = genearate_features(example_df)

In [21]:
example_result.head()

customer_id   month      weekofyear           num1                      \
              nunique mean    nunique mean       sum       max       min   
0       80014       1    9          1   35 -1.456827 -1.456827 -1.456827   
1      146361       1    9          1   35 -0.518679 -0.518679 -0.518679   
2      157857       1    8          1   31 -2.061687 -2.061687 -2.061687   
3      159772       1   12          1   48 -0.276558 -0.276558 -0.276558   
4      180838       1    4          1   13  0.415853  0.415853  0.415853   

            customer_id  
       mean     nunique  
0 -1.456827           1  
1 -0.518679           1  
2 -2.061687           1  
3 -0.276558           1  
4  0.415853           1

In [23]:
import numpy as np
# 100行2列の [0,1) の一様分布乱数のデータフレーム
df = pd.DataFrame(
    np.random.rand(100, 2),
    columns=[f"f_{i}" for i in range(1, 3)]
)
df.head()

,f_1,f_2
0,0.435921,0.280343
1,0.795443,0.049456
2,0.367548,0.266988
3,0.801111,0.919749
4,0.567674,0.780339


In [25]:
# 2次の多項式特徴量（a, b, ab, a^2 b^2）を作成
from sklearn import preprocessing
pf = preprocessing.PolynomialFeatures(
    degree=2,
    interaction_only=False,
    include_bias=False
)
pf.fit(df)

PolynomialFeatures(include_bias=False)

In [28]:
poly_feats = pf.transform(df)
type(poly_feats)

numpy.ndarray

In [30]:
poly_feats.shape

(100, 5)

In [31]:
# データフレーム作成
num_feats = poly_feats.shape[1]
df_transformed = pd.DataFrame(poly_feats, columns=[f"f_{i}" for i in range(1, num_feats + 1)])
df_transformed.head()

,f_1,f_2,f_3,f_4,f_5
0,0.435921,0.280343,0.190027,0.122208,0.078592
1,0.795443,0.049456,0.632730,0.039340,0.002446
2,0.367548,0.266988,0.135092,0.098131,0.071283
3,0.801111,0.919749,0.641779,0.736821,0.845938
4,0.567674,0.780339,0.322253,0.442978,0.608928
